In [ ]:
import pycolmap
from pathlib import Path
import numpy as np
from collections import defaultdict
import h5py
import matplotlib.pyplot as plt
import seaborn as sns
from hloc.utils.io import list_h5_names, get_matches, get_keypoints
from hloc.visualization import plot_images, plot_keypoints, plot_matches, read_image, add_text, cm_RdGn

In [ ]:
DIR = "../image-matching-challenge-2023"
MODE = "train"
NAME = "sift+NN"
dataset = "heritage"
scene = "cyprus"

In [ ]:
scene_dir = Path(f"../outputs/{NAME}/{dataset}/{scene}")

images = Path(f"{DIR}/{MODE}/{dataset}/{scene}/images")

features = scene_dir / "features.h5"
matches = scene_dir / "matches.h5"

In [ ]:
image_names = sorted(list_h5_names(features))
pairs = sorted(list_h5_names(matches))

In [ ]:
plot_images([read_image(images / imname) for imname in image_names[:4]])

In [ ]:
match_matrix = -np.ones([len(image_names), len(image_names)])
for pair in pairs:
    name0, name1 = pair.split("/")
    idx0, idx1 = image_names.index(name0), image_names.index(name1)
    m, sc = get_matches(matches, name0, name1)
    match_matrix[idx0, idx1] = match_matrix[idx1, idx0] = m.shape[0]

ax = sns.heatmap(match_matrix, linewidth=0.0, cmap="hot", mask=match_matrix < 0)

In [ ]:
total = (match_matrix * (match_matrix > 0)).sum(-1)
ax = sns.barplot(x=list(range(len(image_names))), y=total)
ax.set_title('total number of matches per image')

In [ ]:
# 4 images with least amount of matches
ksmallest = np.argsort(total)[:4]
fig = plot_images([read_image(images / image_names[j]) for j in ksmallest], titles=[image_names[j] for j in ksmallest])
[add_text(i, f'matches: {total[j]}') for i, j in enumerate(ksmallest)];

In [ ]:
# print top-pair for each of above image
for j in ksmallest:
    name0 = image_names[j]
    name1 = image_names[np.argmax(match_matrix[j])]
    plot_images([read_image(images / name0), read_image(images / name1)])
    kp0, kp1 = get_keypoints(features, name0), get_keypoints(features, name1)
    m, sc = get_matches(matches, name0, name1)
    plot_matches(kp0[m[:,0]], kp1[m[:,1]])
    add_text(0, f'num matches: {m.shape[0]}')
    add_text(0, name0, pos=(0.01, 0.05))
    add_text(1, name1, pos=(0.01, 0.05))